In [10]:
import pandas as pd
from functools import reduce

In [11]:
df_unemployment =       pd.read_csv('./01_raw_data/행정구역_시도__연령별_실업률_20250806081739.csv', encoding='cp949')
df_rate =               pd.read_csv('./01_raw_data/EV_project/경제 지표/기준금리.csv', encoding='cp949')
df_cpi =                pd.read_csv('./01_raw_data/지출목적별_소비자물가지수_품목포함__2020100__20250806082114.csv', encoding='cp949') 
df_income_total =       pd.read_csv('./01_raw_data/EV_project/경제 지표/지역별_자산__부채__소득_현황_20250806100101.csv', encoding='cp949')

In [12]:
tmp = df_cpi['시점'].astype(str).str.split('.', expand=True)
df_cpi['년도'] = tmp[0].astype(int)
df_cpi['월'] = tmp[1].apply(lambda m: 10 if m == '1' else int(m))

df_cpi = df_cpi.rename(columns={
    '시도별': '지역',
    '데이터': 'cpi'
})
region_map = {
    '전국': '전국',
    '서울특별시': '서울',
    '부산광역시': '부산',
    '대구광역시': '대구',
    '인천광역시': '인천',
    '광주광역시': '광주',
    '대전광역시': '대전',
    '울산광역시': '울산',
    '세종특별자치시': '세종',
    '경기도': '경기',
    '강원특별자치도': '강원',
    '충청북도': '충북',
    '충청남도': '충남',
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '제주특별자치도': '제주'
}
df_cpi['지역'] = df_cpi['지역'].replace(region_map)

df_cpi_sel = df_cpi[['년도','월','지역','cpi']]

df_cpi_sel


,년도,월,지역,cpi
0,2020,1,전국,100.09
1,2020,1,서울,99.80
2,2020,1,부산,100.06
3,2020,1,대구,100.10
4,2020,1,인천,99.94
...,...,...,...,...
1075,2024,12,전북,115.07
1076,2024,12,전남,115.58
1077,2024,12,경북,115.46
1078,2024,12,경남,115.00


In [13]:
tmp = df_unemployment['시점'].astype(str).str.split('.', expand=True)
df_unemployment['년도'] = tmp[0].astype(int)
df_unemployment['분기'] = tmp[1].str.split('/', expand=True)[0].astype(int)


months_map = {1:[1,2,3], 2:[4,5,6], 3:[7,8,9], 4:[10,11,12]}
df_unemployment['월'] = df_unemployment['분기'].map(months_map)

df_unemployment = df_unemployment.explode('월').reset_index(drop=True)

df_unemployment = df_unemployment.drop(columns='분기')

df_unemployment = df_unemployment.rename(columns={
    '시도별': '지역',
    '데이터': '실업률'
})

region_map = {
    '전국': '전국','서울특별시': '서울','부산광역시': '부산',
    '대구광역시': '대구','인천광역시': '인천','광주광역시': '광주',
    '대전광역시': '대전','울산광역시': '울산','세종특별자치시': '세종',
    '경기도': '경기','강원특별자치도': '강원','충청북도': '충북','충청남도': '충남',
    '전라북도': '전북','전라남도': '전남','경상북도': '경북','경상남도': '경남',
    '제주도': '제주','계':'전국','강원도': '강원'
}
df_unemployment['지역'] = df_unemployment['지역'].replace(region_map)


unemp_cols = ['년도','월','지역','실업률']
df_unemp_sel = df_unemployment[unemp_cols]

df_unemp_sel 



,년도,월,지역,실업률
0,2020,1,전국,4.2
1,2020,2,전국,4.2
2,2020,3,전국,4.2
3,2020,4,전국,4.4
4,2020,5,전국,4.4
...,...,...,...,...
1075,2024,8,제주,1.9
1076,2024,9,제주,1.9
1077,2024,10,제주,2.6
1078,2024,11,제주,2.6


In [14]:
df_rate

,변경일자,Unnamed: 1,기준금리
0,2025,05월 29일,2.50
1,2025,02월 25일,2.75
2,2024,11월 28일,3.00
3,2024,10월 11일,3.25
4,2023,01월 13일,3.50
5,2022,11월 24일,3.25
6,2022,10월 12일,3.00
7,2022,08월 25일,2.50
8,2022,07월 13일,2.25
9,2022,05월 26일,1.75


In [15]:

df_rate['date'] = pd.to_datetime(
    df_rate['변경일자'].astype(str) + ' ' + df_rate['Unnamed: 1'],
    format='%Y %m월 %d일'
)
df_rate = df_rate[['date', '기준금리']].sort_values('date')

daily_rate = (
    df_rate
    .set_index('date')
    .resample('D')
    .ffill()
    .reset_index()
)

monthly_rate = (
    daily_rate
    .groupby(daily_rate['date'].dt.to_period('M'))
    .agg({'기준금리': 'mean'})
    .reset_index()
)
monthly_rate['년도'] = monthly_rate['date'].dt.year
monthly_rate['월']   = monthly_rate['date'].dt.month
monthly_rate = monthly_rate[['년도', '월', '기준금리']].rename(columns={'기준금리': '평균기준금리'})

monthly_rate

,년도,월,평균기준금리
0,1999,5,4.750000
1,1999,6,4.750000
2,1999,7,4.750000
3,1999,8,4.750000
4,1999,9,4.750000
...,...,...,...
308,2025,1,3.000000
309,2025,2,2.964286
310,2025,3,2.750000
311,2025,4,2.750000


In [16]:

df_merged_1 = pd.merge(
    df_cpi_sel,
    df_unemp_sel,
    on=['년도', '월','지역'],
    how='left'
)
df_merged_2= pd.merge(
    df_merged_1,
    monthly_rate,
    on=['년도', '월'],
    how='left'
)

df_merged_2.to_csv('./02_interim_data/cpi_unemp_rate.csv', index=False, encoding='utf-8-sig')
df_merged_2

,년도,월,지역,cpi,실업률,평균기준금리
0,2020,1,전국,100.09,4.2,1.25
1,2020,1,서울,99.80,4.5,1.25
2,2020,1,부산,100.06,4.1,1.25
3,2020,1,대구,100.10,3.9,1.25
4,2020,1,인천,99.94,4.6,1.25
...,...,...,...,...,...,...
1075,2024,12,전북,115.07,3.4,3.00
1076,2024,12,전남,115.58,3.5,3.00
1077,2024,12,경북,115.46,2.3,3.00
1078,2024,12,경남,115.00,2.6,3.00
